<a href="https://colab.research.google.com/github/FranciscoBPereira/AnaliseDados_2425_MEI_ISEC/blob/main/AD2425_Aula11_AutoEncoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Setup, Version check and Common imports

# Python ≥3.8 is required
import sys
assert sys.version_info >= (3, 5)


# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

from tensorflow import keras
from tensorflow.keras import layers

# to make this notebook's output stable across runs
np.random.seed(42)

import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

print('Python version: ', sys.version_info)
print('TF version: ', tf.__version__)
print('Keras version: ', keras.__version__)
print('GPU is', 'available' if tf.config.list_physical_devices('GPU') else 'NOT AVAILABLE')

**Dataset Fetching and Preprocessing**

In [ ]:
# Get Fashion MNIST data from Keras datasets
# In this task, targets are irrelevant. Therefore we just keep the 28*28 grayscale images

from tensorflow.keras.datasets import fashion_mnist

(x_train, _), (x_test, _) = fashion_mnist.load_data()

In [ ]:
# Normalize images and adjust shape for autoencoder input

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

print(x_train.shape)
print(x_test.shape)

In [ ]:
# Add noise to images.
# The parameter noise_factor defines the strength of the perturbation
# https://www.tensorflow.org/api_docs/python/tf/random/normal

# After the noise perturbation, the values still belong to the interval [0, 1]

noise_factor = 0.2

x_train_noisy = x_train + noise_factor * tf.random.normal(shape=x_train.shape)
x_test_noisy = x_test + noise_factor * tf.random.normal(shape=x_test.shape)

x_train_noisy = tf.clip_by_value(x_train_noisy, clip_value_min=0., clip_value_max=1.)
x_test_noisy = tf.clip_by_value(x_test_noisy, clip_value_min=0., clip_value_max=1.)

In [ ]:
# Visualize a few images before and after the perturbation
# Change the value of start to inspect other images

start= 0

n = 10
plt.figure(figsize=(20, 6))
for i in range(n):
    ax = plt.subplot(2, n, i + 1)
    plt.title("Originals")
    plt.imshow(tf.squeeze(x_test[i+start]))
    plt.gray()

    ax = plt.subplot(2, n, i + n + 1)
    plt.title("Noisy")
    plt.imshow(tf.squeeze(x_test_noisy[i+start]))
    plt.gray()

plt.show()

**Creation of the Denoising AutoEncoder**

In [ ]:
# Denoise is a subclass of the generic Model class
# Complete the Decoder section. You should use Conv2DTranspose layers to reconstruct (upsample) the latent representation to the original image dimensions
# https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2DTranspose
# https://keras.io/api/models/model/#model-class


from tensorflow.keras.models import Model

class Denoise(Model):
  def __init__(self):
    super(Denoise, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Input(shape=(28, 28, 1)),
      layers.Conv2D(16, (3, 3), activation='relu', padding='same', strides=2),
      layers.Conv2D(8, (3, 3), activation='relu', padding='same', strides=2)])

    self.decoder = tf.keras.Sequential([
      layers.Conv2DTranspose(8, (3,3), activation='relu', padding='same', strides=2),
      layers.Conv2DTranspose(16, (3,3), activation='relu', padding='same', strides=2),
      layers.Conv2D(1, kernel_size=(3, 3), activation='sigmoid', padding='same')])

  def call(self, x):                    # When called with parameter x
    encoded = self.encoder(x)           # The encoder creates a latent representation of x
    decoded = self.decoder(encoded)     # Then, the decoder takes the latent representation and obtains x
    return decoded

In [ ]:
# Create an Autoencoder object

tf.keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

autoencoder = Denoise()

**Compile and Train**

In [ ]:
# Model compilation
# This is a regression problem, where the loss corresponds to the differencde between the input (noisy image) and the output (original image)
# Therefore, the Autoencoder must remove the noise from the input , aiming at delivering a clear image

autoencoder.compile(optimizer='adam', loss=keras.losses.MeanSquaredError())

In [ ]:
# Training

autoencoder.fit(x_train_noisy, x_train,
                epochs=10,
                shuffle=True)

In [ ]:
# Confirm the reduction of dimension from the original image to the latent representation (Encoder)
# Confirm the expansion from the latent representation to the original dimension (Decoder)

autoencoder.summary()

print('***Encoder***')
autoencoder.encoder.summary()

print('***Decoder***')
autoencoder.decoder.summary()

**Evaluate Performance**

In [ ]:
# Evaluate performance on the test set

autoencoder.evaluate(x_test_noisy, x_test)

In [ ]:
# Apply trained autoencoder to images from the test set

encoded_imgs = autoencoder.encoder(x_test_noisy).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

In [ ]:
# Visualize all transformations
# 1. Original images
# 2. Noise images
# 3. Denoised images


n = 10
plt.figure(figsize=(20, 10))
for i in range(n):

    # display original
    ax = plt.subplot(3, n, i + 1)
    plt.title("Originals")
    plt.imshow(tf.squeeze(x_test[i+start]))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display original + noise
    ax = plt.subplot(3, n, i + n + 1)
    plt.title("Noisy")
    plt.imshow(tf.squeeze(x_test_noisy[i+start]))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    bx = plt.subplot(3, n, i + n*2 + 1)
    plt.title("Denoised")
    plt.imshow(tf.squeeze(decoded_imgs[i+start]))
    plt.gray()
    bx.get_xaxis().set_visible(False)
    bx.get_yaxis().set_visible(False)
plt.show()


**CIFAR10 GrayScale to RGB Autoencoder**

In [ ]:
# Get CIFAR10 data from Keras datasets
# In this task, targets are irrelevant. Therefore we just keep the 32*32 RGB images

from tensorflow.keras.datasets import cifar10

(x_trainC, _), (x_testC, _) = cifar10.load_data()

In [ ]:
x_trainC = x_trainC.astype('float32') / 255.
x_testC = x_testC.astype('float32') / 255.

print(x_trainC.shape)
print(x_testC.shape)

In [ ]:
# Create Grayscale versions of the original images
# Use method rgb_to_grayscale from TensorFlow: https://www.tensorflow.org/api_docs/python/tf/image/rgb_to_grayscale

x_trainC_Gray = tf.image.rgb_to_grayscale(x_trainC)
x_testC_Gray = tf.image.rgb_to_grayscale(x_testC)

In [ ]:
print(x_trainC_Gray.shape)
print(x_testC_Gray.shape)

**Quiz**

The goal of the quiz is to create an AutoEncoder that is able to add color to a grayscale image.

The dataset is already created, as we have RGB and Grayscale versions of the same images.

Adapt the previous AutoEncoder so that is can be applied to this problem. In addition to the mandatory changes, the new AutoEncoder must have to additional CNN layers, both in the Encoder and in the Decoder.

After creating the new AutoEncoder, compile, train and analyze results. Training should be longer, with a minimum of 30 epochs.

In [ ]:
## CODE GOES HERE ###



